# Imports

In [ ]:
# initial setup
import os
import asf_search as asf
from eof.download import download_eofs

# Settings

In [ ]:
# paths
data_dir = '/data'
download_folder = os.path.join(data_dir,'scenes')
credentials_path = "credentials/credentials_earthdata.txt"
precise_orb_download_folder = os.path.join(data_dir,'orbits','POEORB')
restituted_orb_download_folder = os.path.join(data_dir,'orbits','RESORB')

# Credentials

In [ ]:
# read credentials from file
with open(credentials_path, "r") as f:
   txt = str(f.read())
   uid = txt.split('\n')[1].split('login')[-1][1:]
   pswd = txt.split('\n')[2].split('password')[-1][1:]
   email = txt.split('\n')[3].split('email')[-1][1:]

# AOI

In [ ]:
wkt = "POINT (130.7085216 -16.0044816)" # South Australia
print(f'Searching over point: {wkt}')

# Search

In [ ]:
prod = 'GRD_HD'
# prod = 'GRD_MD'
# prod = 'GRD_MS'
# prod = 'GRD_HS'
# prod = 'GRD_FD'
# prod = 'SLC'
# product type https://github.com/asfadmin/Discovery-asf_search/blob/master/asf_search/constants/PRODUCT_TYPE.py
results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                     intersectsWith=wkt, 
                     maxResults=10, 
                     processingLevel=prod,
                     beamMode='IW',
                     start='2021-12-31T11:59:59Z',
                     end='2022-01-19T11:59:59Z')
print(len(results))

In [ ]:
for r in results:
    print(r.__dict__['umm']['GranuleUR'].split('-')[0])
    print(r.__dict__['umm']['AdditionalAttributes'][1]['Values'],
          r.__dict__['umm']['AdditionalAttributes'][17],
          r.__dict__['umm']['OrbitCalculatedSpatialDomains'][0]) #[1] #['FRAME_NUMBER']

# Download Scenes

In [ ]:
# download
session = asf.ASFSession()
session.auth_with_creds(uid,pswd)

In [ ]:
# download all results
scene_paths = []
scene_names = []
for scene in results:
    name = scene.__dict__['umm']['GranuleUR'].split('-')[0]
    scene_names.append(name)
    print(name)
    path = os.path.join(download_folder, name)
    scene.download(path=download_folder, session=session)
    scene_paths.append(path)

# Download Precise Orbit Files

In [ ]:
for scene in  scene_paths:
    # download precise orbits
    download_eofs(sentinel_file=scene, save_dir=precise_orb_download_folder, orbit_type='precise')
    # download restituted orbits
    download_eofs(sentinel_file=scene, save_dir=restituted_orb_download_folder, orbit_type='restituted')

In [ ]:
print('precise orbits')
for p in os.listdir(precise_orb_download_folder):
    print(os.path.join(precise_orb_download_folder,p))

print('restited orbits')
for p in os.listdir(restituted_orb_download_folder):
    print(os.path.join(restituted_orb_download_folder,p))